In [1]:
from visualize import * 

In [2]:
from PySudoku import *

In [3]:
from solution_test import *

In [4]:
assignments = []
rows = 'ABCDEFGHI'
cols = '123456789'

def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """

    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

In [159]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    for peers in unitlist:
        naked_twin = [box for box in peers if len(values[box]) == 2]
        naked_twin_digits = [values[box] for box in peers if len(values[box]) == 2]
        naked_twin_values = [twin for twin in naked_twin if naked_twin_digits.count(values[twin]) > 1]
        for twin in naked_twin_values:
            twin_val = values[twin]
            print("val", twin_val, twin )
            for digit in twin_val:
                for peer in peers:
                    if digit in values[peer] and values[peer] != twin_val:
                        print("digit", digit, values[peer] )
                        values[peer] = values[peer].replace(digit,"")

    return values

In [161]:
def cross(a, b):
    return [s+t for s in a for t in b]

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diag1 = [[str(r)+str(10- int(c)) for (r,c) in zip(rows,cols)]]

diag2 = [[r+c for (r,c) in zip(rows,cols)]]
#print(diag2)
unitlist = row_units + column_units + square_units + diag1 + diag2
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
#print('units',units)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
#print(peers.keys())



In [7]:
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

In [8]:
def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [9]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

In [10]:
def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [11]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = naked_twins(values)
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [12]:
def search(values):
    values = reduce_puzzle(values)
    #print(values)
    if values is False:
        return False ## Failed earlier
    
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    #print('s',s)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        #print(new_sudoku)
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [13]:
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
display(grid_values(diag_sudoku_grid))

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

In [160]:
display(search(grid_values(diag_sudoku_grid)))

val 57 B2
digit 5 457
digit 7 47
val 57 F6
val 57 B2
val 57 F6
val 57 B2
val 57 F6
val 57 B2
val 57 F6
val 57 B2
val 57 F6
val 19 H1
val 19 H6
val 57 B2
val 57 F6
val 19 H1
val 19 H6
val 89 B1
digit 8 3789
digit 8 3789
digit 9 379
digit 9 379
val 89 B9
val 19 H1
val 19 H6
val 89 F3
digit 8 3789
digit 9 379
val 89 I3
val 35 D5
digit 3 23
digit 3 37
val 35 F5
val 37 A3
val 89 B1
val 37 B3
val 89 C2
val 35 D5
val 35 F5
val 89 B1
val 37 B3
val 37 B4
val 89 B9
val 19 H1
val 19 H6
val 19 I2
digit 9 89
val 19 I7
val 37 A3
val 37 B3
val 35 D5
val 35 F5
val 89 B9
digit 9 29
digit 9 59
val 89 F9
val 37 A3
val 37 B3
val 35 D5
val 35 F5
val 19 H1
val 19 I2
val 19 G8
val 19 I7
val 37 B3
val 37 B4
val 15 D1
val 15 F1
val 37 A3
val 37 B3
val 35 D5
val 35 F5
val 37 A3
val 37 B3
val 15 D1
val 15 F1
val 35 D5
val 35 F5
val 35 D5
val 35 F5
val 35 D5
val 35 F5
2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
